***En esta entrega, seguiremos trabajando con la API de transporte de la Ciudad de Buenos Aires. Nos enfocaremos en normalizar la información e insertarla en una tabla que disponga de clave primaria compuesta.***

* 
    API Doc: https://api-transporte.buenosaires.gob.ar/console
*   API Console: https://apitransporte.buenosaires.gob.ar/console/


**Librerías a utilizar**

In [1]:
import psycopg2 #Conexión con redshift
import requests #Para conectar con la api
import datetime #Manejo de fechas
import pandas as pd
import os
from dotenv import load_dotenv

**Funcion encargada de conectar con la API**

In [2]:
def consulta_api_params(url,params):
    try:
        response = requests.get(url,params = params)
        if response.status_code == 200:  #Corroboramos que la peticion sea exitosa
            print("Respuesta de la API:")
            # Convertimos la respuesta a JSON
            return response.json()
        else:
            print("Error al consultar la API. Código de estado:", response.status_code)
    except Exception as e:
        print("Error al conectarse a la API:", e)

**Funcion que nos permite pasarle los diferentes parametros a la consulta.**

In [3]:
def get_params(route_id,agency_id,trip,client_id,client_secret):
    params = {}
    
    if(route_id != ""):
        params['route_id'] = route_id
    
    if(agency_id != ""):
        params['agency_id'] = agency_id
        
    if(trip != ""):
        params['Trip'] = trip
        
    if(client_id != ""):
        params['client_id'] = client_id
        
    if(client_secret != ""):
        params['client_secret'] = client_secret
        
    return params

**Funcion encargada de corregir valores que por defecto vienen con valores incompatibles**

In [4]:
def correcion_valores(docjson):
    df = pd.DataFrame(docjson)
    df['datetime_consulta'] = pd.to_datetime(df['timestamp'],unit='s') - datetime.timedelta(hours=3)
    df['speed'] = df['speed'].round(3)
    df['timestamp'] = df['timestamp'].astype(str)
    df['direction'] = df['direction'].astype(str)
    df['agency_id'] = df['agency_id'].astype(str)
    return df

**Seteamos las credenciales y parametros para la conexión**

In [5]:
url_p = 'https://apitransporte.buenosaires.gob.ar/colectivos/vehiclePositionsSimple'
load_dotenv()
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

params = get_params("","","",client_id,client_secret)

datos_json = consulta_api_params(url_p,params = params)
# Crear DataFrame a partir del JSON
df = correcion_valores(datos_json)



Respuesta de la API:


**Revisamos tipos de datos del dataframe obtenido**

In [6]:
print(df.dtypes)

route_id                     object
latitude                    float64
longitude                   float64
speed                       float64
timestamp                    object
id                           object
direction                    object
agency_name                  object
agency_id                    object
route_short_name             object
tip_id                       object
trip_headsign                object
datetime_consulta    datetime64[ns]
dtype: object


**Reordenamos el Dataframe para su posterior inserción en nuestra tabla en Redshift**

In [7]:
new_order = ['id','datetime_consulta','route_id','latitude','longitude','speed','timestamp','direction','agency_name','agency_id','route_short_name','tip_id','trip_headsign']

df = df[new_order]

print("Dataframe Reordenado")
display(df)

Dataframe Reordenado


,id,datetime_consulta,route_id,latitude,longitude,speed,timestamp,direction,agency_name,agency_id,route_short_name,tip_id,trip_headsign
0,1855,2024-04-22 23:28:48,1468,-34.70196,-58.670100,0.000,1713839328,0,MICROOMNIBUS SAAVEDRA S.A.T.A.C.I.,82,153A,95462-1,a Cañuelas
1,1856,2024-04-22 23:28:48,1467,-34.68598,-58.662500,0.000,1713839328,1,MICROOMNIBUS SAAVEDRA S.A.T.A.C.I.,82,321A,95372-1,a Chacarita x Camarones
2,1867,2024-04-22 23:28:48,1466,-34.65423,-58.645900,6.389,1713839328,0,MICROOMNIBUS SAAVEDRA S.A.T.A.C.I.,82,321A,95324-1,a Autodromo x Camarones
3,1877,2024-04-22 23:28:48,316,-34.79680,-58.253600,5.000,1713839328,1,EL NUEVO HALCON S.A.,63,148D,24216-1,a Constitución
4,1885,2024-04-22 23:28:48,1648,-34.69716,-58.326435,0.000,1713839328,0,EL NUEVO HALCON S.A.,63,148I,106489-1,G x San Martí­n
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2767,69649,2024-04-22 23:28:48,1420,-34.82198,-58.362705,10.833,1713839328,0,SAN VICENTE S.A. DE TRANSPORTES,68,79G,91703-1,a Munro x Viaducto Carranza
2768,69749,2024-04-22 23:28:46,1776,-34.72075,-58.570114,0.000,1713839326,0,LA VECINAL DE MATANZA SACI. DE MICROOMNIBUS,76,180A,115565-1,D - Av. Congreso y Bucarelli
2769,69757,2024-04-22 23:28:48,1068,-34.71371,-58.561800,12.222,1713839328,1,LA VECINAL DE MATANZA SACI. DE MICROOMNIBUS,76,630R2,69497-1,a Canal San Fernando
2770,69760,2024-04-22 23:28:48,1778,-34.69710,-58.536835,10.000,1713839328,0,LA VECINAL DE MATANZA SACI. DE MICROOMNIBUS,76,180B,115785-1,A - Florida


**Función encargada de cargar los datos obtenidos en la tabla de Redshift**

In [8]:
def cargar_df_redshift(df):
    try:
        conection = psycopg2.connect(
        dbname = os.getenv('NAME_DATABASE'),
        user = os.getenv('USER_DATABASE'),
        password = os.getenv('PASS_DATABASE'),
        host = os.getenv('HOST_DATABASE'),
        port= os.getenv('PORT_DATABASE'))
        
        
        datetime_insert = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") #Campo de fecha de insercion en la tabla
        cursor = conection.cursor()
        
        for i in range(df.shape[0]-1):
            tupla_final = tuple(df.iloc[i]) + (datetime_insert,)
            cursor.execute("INSERT INTO cristiangen16_coderhouse.Fact_Bus_Position(idTrack, dttmCatch,idRoute ,adLatitude,adLongitude,adSpeed,adTimestamp,adDirection,adAgencyName,idAgency,adRouteShortName,idTransport,adTripHeadsign,dttmInsert) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)" , tupla_final)
        
        conection.commit()
        print("Datos cargados en Amazon Redshift con éxito.")
        cursor.close()
        conection.close()
        
    except Exception as e:
        print("Error al cargar datos en Redshift:", e)

**Invocamos a la función para cargar los datos**

In [ ]:
cargar_df_redshift(df)